<a href="https://colab.research.google.com/github/DaniloTovar/Blockchain1/blob/main/Bitcoin_blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import hashlib as hlib
import itertools

## Funciones adicionales


In [9]:
def hash(value):
  """
  Encrypts 'value' using a SHA-256 algorithm and returns the result
  """
  assert type(value) == str, "Wrong input type, 'value' should be a String"
  return hlib.sha256(value.encode()).hexdigest()

def hashList(input):
  """
  Encrypts each element of 'input' using a SHA-256 algorithm
  """
  assert type(input) == list, "Wrong input type, 'input' should be a List"
  aux = []
  for i in input:
    try:
      aux.append(hash(i))
    except(AttributeError, TypeError):
      raise AssertionError("List should only contain Strings")
  return aux

def getRoot(transactions):
  """
  Calculates the Merkle Root from a list 'transactions' using a SHA-256 algorithm
  """
  assert type(transactions) == list, "Wrong input type, 'transactions' should be a list"
  assert len(transactions) != 0, "List of transactions cannot be empty"

  while(len(transactions)!= 1):
    length = len(transactions)
    next = []

    if(np.mod(length,2)):
      last = transactions[-1]
      transactions = transactions.append(last)

    for i in range(int(length/2)):
      string = transactions[i+i] + transactions[i+i+1]
      hash = hash(string)
      next.append(hash)
      # print(next)

    transactions = next

  return transactions[0]

def genNounce(top):
  """
  Generates a random number between 0 and 'top'
  """
  assert type(top) == int, "Wrong input type, 'top' should be a number"
  return int(np.random.random()*np.random.random()*top)

## Clases a utilizar para la construcción de la blockchain

In [38]:
class Wallet:
  """
  """
  def __init__(self, private):
    self.private = private
    self.public = hash(private)
    self.open_tx = []
    self.balance = 0

  def describe(self):
    print("public: ", self.public,"\n",
          "available transactions: ", self.open_tx,"\n",
          "balance: ", self.balance,"\n")

In [29]:
class Output:
  """
  """
  def __init__(self, amount, to):
    assert type(to) == Wallet, "Wrong destination, should be a 'Wallet'"
    self.amount = amount
    self.to = to
    self.id_tx = -1
    self.spent = False

In [13]:
class Transaction:
  """
  """
  # Id autoincremental comenzando en 1
  id_tx = itertools.count(1)

  # Constructor de una transaccion normal
  def __init__(self, inputs, outputs):
    self.id = next(Transaction.id_tx)

    assert all((type(ins) == Output & ins.id_tx != self.id & ins.spent == False) for ins in inputs), "List of inputs should only contain previous unspent 'Output' objects"
    assert all((type(out) == Output & out.id_tx == -1) for out in outputs), "List of outputs should only contain new 'Output' objects"

    self.inputs = inputs
    self.outputs = outputs
    self.block_hash

    # Actualizamos las wallets
    try:
      # Actualizamos los balance
      for input in self.inputs:
        check = input.to.balance - input.amount
        if(check < 0):
          raise ArithmeticError("Balance cannot be negative")
          break
        input.to.balance -= input.amount

      for output in self.outputs:
        output.to.balance += output.amount

      # Actualizamos la lista de trasacciones abiertas
      for input in self.inputs:
        input.to.open_tx.remove(input.id_tx)
      for output in self.outputs:
        output.to.open_tx.append(self.id)

    except:
      raise ArithmeticError("Sum of operations is invalid")

    for ins in inputs:
      ins.spent = True        # Actualizamos los inputs como 'spent' cuando se genera esta transaccion

    for out in outputs:
      out.id_tx = self.id     # Actualizamos los outputs para relacionarlos a esta transaccion



  # Constructor de un CoinBase
  def __init__(self, wallet):
    self.id = next(Transaction.id_tx)
    self.inputs = 0
    self.outputs = Output(50, wallet)
    self.block_hash

    self.outputs.to.balance += self.outputs.amount  # Actualizamos el balance de la wallet
    self.outputs.to.open_tx.append(self.id)     # Actualizamos la lista de transacciones abiertas relacionadas a la wallet
    self.outputs.id_tx = self.id     # Actualizamos el output para relacionarlo a esta transaccion


In [14]:
class Block:
  """
  """
  def __init__(self, transactions, prev_hash):
    assert all((type(tx) == Transaction) for tx in transactions), "List of transactions should only contain 'Trasaction' objects"

    self.transactions = transactions
    self.merkle_root = getRoot(transactions)
    self.prev_hash = prev_hash
    self.nounce = genNounce(10**10)

    for tx in transactions:
      tx.block_hash = self.genHash()

  def getSelf(self):
    return ("Transactions: ", self.transactions,"\n",
            "Root: ",self.merkle_root, "\n",
            "Previous hash: ",self.prev_hash,"\n",
            "Nounce: ",self.nounce,"\n")

  def getBody(self):
    return ("Transactions: ", self.transactions,"\n")

  def getHeader(self):
    return ("Root: ",self.merkle_root, "\n",
            "Previous hash: ",self.prev_hash,"\n",
            "Nounce: ",self.nounce,"\n")

  def genHash(self):
    str_self = str(self.merkle_root) + str(self.prev_hash) + str(self.nounce)
    return hash(self.toString())

  def Validate(self):
    for tx in self.transactions:
      out_total_amount = sum([].append(out.amount) for out in tx.outputs[1:])   # Suma todos los valores de outputs excepto el CoinBase
      in_total_amount = sum([].append(ins.amount) for ins in tx.inputs[1:])     # Suma todos los valores de inputs excepto el CoinBase
      print(out_total_amount, in_total_amount)
      return in_total_amount >= out_total_amount

In [7]:
class Chain:
  """
  """
  def __init__(self, chain):
    self.chain = chain

  def addBlock(self, block, difficulty_level):
    assert type(difficulty_level) == int, "Wrong type, 'difficulty_level' should be a integer"
    assert type(difficulty_level) >= 0, "'difficulty_level' should not be negative"
    assert type(block) == Block, "Wrong type, 'block' should be a Block"

    assert block.Validate() == True, "Invalid block, sum of transaction operations doesnt match"

    goal = str("0"*difficulty_level)
    hashed_nounce = hash(str(block.nounce))
    while(not(hashed_nounce.startswith(goal))):
      block.nounce = genNounce
      hashed_nounce = hash(str(block.nounce))
    print(goal,"? -->", hashed_nounce, "with", block.nounce)
    self.chain.append(block)
    print("Block added")


## Inicialización de la blockchain

Creación de Wallets vacias

In [39]:
Wallet1 = Wallet("Pepito123456")
Wallet2 = Wallet("Edison147852")
Wallet3 = Wallet("Fernando987")
Wallet4 = Wallet("Ricardo456456")

Wallet1.describe()
Wallet2.describe()
Wallet3.describe()
Wallet4.describe()

public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 available transactions:  [] 
 balance:  0 

public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 available transactions:  [] 
 balance:  0 

public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 available transactions:  [] 
 balance:  0 

public:  5bf8162f54797354bba873f473f0065ce5108e21cee3870d069aba4ad37eaa21 
 available transactions:  [] 
 balance:  0 



Modificacion arbitraria de los 'balace' de las wallets para realizar el ejemplo

In [40]:
Wallet1.balance = 20
Wallet2.balance = 100
Wallet3.balance = 50
Wallet4.balance = 30

Wallet1.describe()
Wallet2.describe()
Wallet3.describe()
Wallet4.describe()

public:  1217d76e89796d523a61815fb7d198db01d6c79386ee0bb6420c3f87fef5e063 
 available transactions:  [] 
 balance:  20 

public:  16831949036b6da4f8d8688af827e64ac3d1e24a218905f81a9d43d6b12cc001 
 available transactions:  [] 
 balance:  100 

public:  de9a6e7558175562e25dba71834875f0f89a5e959dd94f13a1c2f0d91ba6f010 
 available transactions:  [] 
 balance:  50 

public:  5bf8162f54797354bba873f473f0065ce5108e21cee3870d069aba4ad37eaa21 
 available transactions:  [] 
 balance:  30 



Creación de un conjunto de transacciones

Creación de un bloque 'inicial'

Cración de una cadena vacía

In [42]:
long_chain = Chain([])
long_chain.chain

[]

Añadimos el bloque a una cadena

## Simulación de la interacción en la blockchain